In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
from matplotlib import pyplot as plt

In [4]:
data = pd.read_csv('Combined_News_DJIA.csv')

In [5]:
train = data[data['Date'] < '2016-01-01']
test = data[data['Date'] > '2015-12-31']

In [6]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))
basicvectorizer = CountVectorizer(ngram_range=(2,2), max_features=5000)
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)
X_train = basictrain.toarray()
X_train = np.array(X_train)
y_train = np.array(train['Label'])
print(y_train.shape)
print(X_train.shape)

(1863, 5000)
(1863,)
(1863, 5000)


In [8]:
params = {
    'booster': 'gbtree',
    'objective': 'reg:logistic',
    'gamma': 0.2,
    'max_depth': 6,
    'lambda': 2,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'silent': 1,
    'eta': 0.1,
    'seed': 1000,
}
#plst = params.items()
dtrain = xgb.DMatrix(X_train, y_train)
num_rounds = 500
model = xgb.train(params, dtrain, num_rounds)


In [9]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(testheadlines)
print(basictest.shape)
X_test = basictest.toarray()
X_test = np.array(X_test)
y_test = np.array(test['Label'])

(126, 5000)


In [10]:
dtest = xgb.DMatrix(X_test)
ypred = model.predict(dtest)
#print(ypred)
#print(ypred.shape)
threshold = 0.6
ypred[ypred >= threshold] = 1
ypred[ypred < threshold] = 0
#print(ypred[1])

In [11]:
cnt1 = 0
cnt2 = 0
for i in range(len(y_test)):
    if ypred[i] == y_test[i]:
        cnt1 += 1
    else:
        cnt2 += 1

print("Accuracy: %.2f %% " % (100 * cnt1 / (cnt1 + cnt2)))

Accuracy: 51.59 % 


In [17]:
from sklearn.metrics import accuracy_score
param_dist = {'objective':'binary:logistic', 'n_estimators':2}
clf = xgb.XGBClassifier(**param_dist)

clf.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric='logloss',
        verbose=True)

evals_result = clf.evals_result()
y_pred = clf.predict(X_test)
accu_score = accuracy_score(y_test, y_pred)
print(accu_score)

[0]	validation_0-logloss:0.690297	validation_1-logloss:0.692341
[1]	validation_0-logloss:0.68778	validation_1-logloss:0.690872
0.5793650793650794
